<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/Migrate_from_pg_embedding_to_pg_vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://neon.tech/docs/extensions/pg_embedding#migrate-from-pg_embedding-to-pgvector


In [1]:
#Install Libraries to access Google Drive and OpenAI resources.
%pip install colab-env --upgrade --quiet --root-user-action=ignore
import colab_env

  Preparing metadata (setup.py) ... done
Mounted at /content/gdrive


In [ ]:
# install PSQL WITH DEV Libraries AND PG embedding
!apt install postgresql postgresql-contrib &>log
!service postgresql restart
!sudo apt install postgresql-server-dev-all

In [ ]:
!cp -pr /content/gdrive/MyDrive/tools/pg_embedding /content/
%cd /content/pg_embedding/
print()
print('START: PG embedding COMPILATION')
!make
!make install # may need sudo
print('END: PG embedding COMPILATION')
print()

In [4]:
# PostGRES SQL Settings
import psycopg2 as ps

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

!sudo -u postgres psql -c "DROP EXTENSION vector CASCADE"
!sudo -u postgres psql -c "DROP EXTENSION embedding CASCADE"

!sudo -u postgres psql -c "CREATE EXTENSION embedding"

!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id BIGSERIAL PRIMARY KEY, embedding real[])"

h="{1,2,3}"
hh= "INSERT INTO documents(id, embedding) VALUES (1,'%s'), (2,'{4,5,6}')"%h
print(hh)

def insert_document(id,embedding):
    #review_embedding=get_embedding(text)
    ### INSERT INTO DB
    DB_NAME = "postgres"
    DB_USER = "postgres"
    DB_PASS = "postgres"
    DB_HOST = "localhost"
    DB_PORT = "5432"
    conn = ps.connect(database=DB_NAME,
							user=DB_USER,
							password=DB_PASS,
							host=DB_HOST,
							port=DB_PORT)


    cur = conn.cursor() # creating a cursor

    cur.execute("""
        INSERT INTO documents
        (id, embedding)
        VALUES ('%s',
                '%s')""" % (id,embedding))

    conn.commit()
    print("INSERT EMBEDDING %s successfully"%embedding)
    conn.close()
    cur.close()


insert_document(1,'{1,2,3}')
insert_document(2,'{4,5,6}')


#!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw(embedding) WITH (dims=3, m=3, efconstruction=5, efsearch=5)"

/content
ALTER ROLE
ERROR:  extension "vector" does not exist
ERROR:  extension "embedding" does not exist
CREATE EXTENSION
ERROR:  table "documents" does not exist
CREATE TABLE
INSERT INTO documents(id, embedding) VALUES (1,'{1,2,3}'), (2,'{4,5,6}')
INSERT EMBEDDING {1,2,3} successfully
INSERT EMBEDDING {4,5,6} successfully


In [5]:
!sudo -u postgres psql -c "DROP EXTENSION embedding CASCADE"

DROP EXTENSION


In [6]:
# install PSQL WITH DEV Libraries AND PG embedding
!apt install postgresql postgresql-contrib &>log
!service postgresql restart
!sudo apt install postgresql-server-dev-all

!cp -pr /content/gdrive/MyDrive/tools/pgvector /content/
%cd /content/pgvector/
print()
print('START: PG VECTOR COMPILATION')
!make
!make install
print('END: PG VECTOR COMPILATION')
print()

 * Restarting PostgreSQL 14 database server
   ...done.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
postgresql-server-dev-all is already the newest version (238).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
/content/pgvector

START: PG VECTOR COMPILATION
make: Nothing to be done for 'all'.
/bin/mkdir -p '/usr/lib/postgresql/14/lib'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/bin/mkdir -p '/usr/share/postgresql/14/extension'
/usr/bin/install -c -m 755  vector.so '/usr/lib/postgresql/14/lib/vector.so'
/usr/bin/install -c -m 644 .//vector.control '/usr/share/postgresql/14/extension/'
/usr/bin/install -c -m 644 .//sql/vector--0.1.0--0.1.1.sql .//sql/vector--0.1.1--0.1.3.sql .//sql/vector--0.1.3--0.1.4.sql .//sql/vector--0.1.4--0.1.5.sql .//sql/vector--0.1.5--0.1.6.sql .//sql/vector--0.1.6--0.1.7.sql .//sql/vector--0.1.7--0.1.8.sql .//sql/vector--0.1.8--0.2.0.sql .//sql/vector--0.2.0--0.2.1.sql .//sql/vector--0

In [7]:
# PostGRES SQL Settings
import psycopg2 as ps

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

!sudo -u postgres psql -c "CREATE EXTENSION vector"

!sudo -u postgres psql -c "DROP TABLE documents"
!sudo -u postgres psql -c "CREATE TABLE documents(id BIGSERIAL PRIMARY KEY, embedding vector(3))"

insert_document(1,'[1,2,3]')
insert_document(2,'[4,5,6]')
!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw (embedding vector_l2_ops)"

#!sudo -u postgres psql -c "CREATE INDEX ON documents USING hnsw(embedding ann_cos_ops) WITH (m=3)"


/content
ALTER ROLE
CREATE EXTENSION
DROP TABLE
CREATE TABLE
INSERT EMBEDDING [1,2,3] successfully
INSERT EMBEDDING [4,5,6] successfully
CREATE INDEX


In [8]:
def query_selection(query):
      # PostGRES SQL Settings
      import psycopg2 as ps
      DB_NAME = "postgres"
      DB_USER = "postgres"
      DB_PASS = "postgres"
      DB_HOST = "localhost"
      DB_PORT = "5432"
      conn = ps.connect(database=DB_NAME,
                user=DB_USER,
                password=DB_PASS,
                host=DB_HOST,
                port=DB_PORT)
      cur = conn.cursor() # creating a cursor
      cur.execute("""
          %s """%query)
      records = cur.fetchall()
      print("Total rows are:  ", len(records))
      print("Printing each row")
      print()
      n=0
      for row in records:
          n=n+1
          print("ROW %s: "%n, row)
      conn.close()
      cur.close()
      print()
      print("QUERY SELECTION successfully")
      print()

In [9]:
## <-> and <=> Euclidean (L2) and Cosine distances NO Manhattan distance: <~>
query_selection("SELECT * FROM documents ORDER BY embedding::vector <-> '[3,1,2]' LIMIT 5")

Total rows are:   2
Printing each row

ROW 1:  (1, '[1,2,3]')
ROW 2:  (2, '[4,5,6]')

QUERY SELECTION successfully



In [10]:
# Euclidean (L2) distance index:
#CREATE INDEX ON documents USING hnsw(embedding) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <-> array[3,3,3] LIMIT 1;

# Cosine distance index:
#CREATE INDEX ON documents USING hnsw(embedding ann_cos_ops) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <=> array[3,3,3] LIMIT 1;

# Manhattan distance index:
#CREATE INDEX ON documents USING hnsw(embedding ann_manhattan_ops) WITH (dims=3, m=3, efconstruction=5, efsearch=5);
#SET enable_seqscan = off;
#SELECT id FROM documents ORDER BY embedding <~> array[3,3,3] LIMIT 1;



# AWS

In [5]:
#!rm -rf /content/*.pdf
!mkdir -p /content/data/
%cd /content/data/

from urllib.request import urlretrieve
urls = [
    'https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/2022-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/2021-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Shareholder-Letter-and-1997-Shareholder-Letter.pdf',
    'https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Shareholder-Letter.pdf'
]

filenames = [
    'AMZN-2022-Shareholder-Letter.pdf',
    'AMZN-2021-Shareholder-Letter.pdf',
    'AMZN-2020-Shareholder-Letter.pdf',
    'AMZN-2019-Shareholder-Letter.pdf'
]

metadata = [
    dict(year=2022, source=filenames[0]),
    dict(year=2021, source=filenames[1]),
    dict(year=2020, source=filenames[2]),
    dict(year=2019, source=filenames[3])]

data_root = "/content/data/"

for idx, url in enumerate(urls):
    file_path = data_root + filenames[idx]
    #print(file_path)
    urlretrieve(url, file_path)

/content/data


In [ ]:
%pip install openai==0.28  --root-user-action=ignore
%pip install langchain
%pip install "unstructured[all-docs]"
%pip install tiktoken
%pip install -q -U sentence-transformers

In [6]:
from pypdf import PdfReader, PdfWriter
import glob

local_pdfs = glob.glob(data_root + '*.pdf')

for local_pdf in local_pdfs:
    pdf_reader = PdfReader(local_pdf)
    pdf_writer = PdfWriter()
    for pagenum in range(len(pdf_reader.pages)-3):
        page = pdf_reader.pages[pagenum]
        pdf_writer.add_page(page)

    with open(local_pdf, 'wb') as new_file:
        new_file.seek(0)
        pdf_writer.write(new_file)
        new_file.truncate()

In [7]:
import numpy as np
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

#%cd /content/data

documents = []

for idx, file in enumerate(filenames):
    loader = PyPDFLoader(data_root + file)
    document = loader.load()
    for document_fragment in document:
        document_fragment.metadata = metadata[idx]

    documents += document

# - in our testing Character split works better with this PDF data set
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 512,
    chunk_overlap  = 100,
)

docs = text_splitter.split_documents(documents)

print(f'# of Document Pages {len(documents)}')
print(f'# of Document Chunks: {len(docs)}')

# of Document Pages 25
# of Document Chunks: 299


In [2]:
def query_selection(query):
      # PostGRES SQL Settings
      import psycopg2 as ps
      DB_NAME = "postgres"
      DB_USER = "postgres"
      DB_PASS = "postgres"
      DB_HOST = "localhost"
      DB_PORT = "5432"
      conn = ps.connect(database=DB_NAME,
                user=DB_USER,
                password=DB_PASS,
                host=DB_HOST,
                port=DB_PORT)
      cur = conn.cursor() # creating a cursor
      cur.execute("""
          %s """%query)
      records = cur.fetchall()
      print("Total rows are:  ", len(records))
      print("Printing each row")
      print()
      n=0
      for row in records:
          n=n+1
          print("ROW %s: "%n, row)
      conn.close()
      cur.close()
      print()
      print("QUERY SELECTION successfully")
      print()

In [16]:
#Install Libraries to access Google Drive and OpenAI resources.
%pip install colab-env --upgrade --quiet --root-user-action=ignore
import colab_env

!cp -pr /content/gdrive/MyDrive/tools/pg_embedding /content/
%cd /content/pg_embedding/
print()
print('START: PG embedding COMPILATION')
!make
!make install # may need sudo
print('END: PG embedding COMPILATION')
print()

/content/pg_embedding

START: PG embedding COMPILATION
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -g -g -O2 -flto=auto -ffat-lto-objects -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security -fno-omit-frame-pointer -Ofast -fPIC -I. -I./ -I/usr/include/postgresql/14/server -I/usr/include/postgresql/internal  -Wdate-time -D_FORTIFY_SOURCE=2 -D_GNU_SOURCE -I/usr/include/libxml2   -c -o embedding.o embedding.c
g++ -Wall -Wpointer-arith -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -g -g -O2 -flto=auto -ffat-lto-objects -flto=auto -ffat-lto-objects -fstack-protector-strong -Wformat -Werror=format-security -std=c++11 -f

In [17]:
#!apt install postgresql postgresql-contrib &>log
#!service postgresql restart
#!sudo apt install postgresql-server-dev-all

%cd /content/
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

#DROP EXTENSION IF EXISTS
!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS embedding;"
#!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS vector CASCADE"

!sudo -u postgres psql -c "CREATE EXTENSION embedding;"
#!sudo -u postgres psql -c "CREATE EXTENSION vector;"

query_selection("SELECT name FROM pg_available_extensions order by 1")

/content
ALTER ROLE
NOTICE:  extension "embedding" does not exist, skipping
DROP EXTENSION
CREATE EXTENSION
Total rows are:   47
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql

In [ ]:
!cp -pr /content/gdrive/MyDrive/tools/pgvector /content/
%cd /content/pgvector/
print()
print('START: PG VECTOR COMPILATION')
!make
!make install
print('END: PG VECTOR COMPILATION')
print()

In [3]:
import os
collection_name='AWS'

from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import PGEmbedding

%pip install colab-env
import colab_env

connection_string = os.getenv("DATABASE_URL")

# https://supabase.com/blog/fewer-dimensions-are-better-pgvector
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

#https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

%cd /content/

# install PSQL WITH DEV Libraries AND PG embedding
#!apt install postgresql postgresql-contrib &>log
#!service postgresql restart
#!sudo apt install postgresql-server-dev-all

!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

#DROP EXTENSION IF EXISTS
!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS embedding CASCADE"
#!sudo -u postgres psql -c "DROP EXTENSION IF EXISTS vector CASCADE"

!sudo -u postgres psql -c "CREATE EXTENSION embedding;"
#!sudo -u postgres psql -c "CREATE EXTENSION vector;"

query_selection("SELECT name FROM pg_available_extensions order by 1")

/content
ALTER ROLE
DROP EXTENSION
CREATE EXTENSION
Total rows are:   47
Printing each row

ROW 1:  ('adminpack',)
ROW 2:  ('amcheck',)
ROW 3:  ('autoinc',)
ROW 4:  ('bloom',)
ROW 5:  ('btree_gin',)
ROW 6:  ('btree_gist',)
ROW 7:  ('citext',)
ROW 8:  ('cube',)
ROW 9:  ('dblink',)
ROW 10:  ('dict_int',)
ROW 11:  ('dict_xsyn',)
ROW 12:  ('earthdistance',)
ROW 13:  ('embedding',)
ROW 14:  ('file_fdw',)
ROW 15:  ('fuzzystrmatch',)
ROW 16:  ('hstore',)
ROW 17:  ('insert_username',)
ROW 18:  ('intagg',)
ROW 19:  ('intarray',)
ROW 20:  ('isn',)
ROW 21:  ('lo',)
ROW 22:  ('ltree',)
ROW 23:  ('moddatetime',)
ROW 24:  ('old_snapshot',)
ROW 25:  ('pageinspect',)
ROW 26:  ('pg_buffercache',)
ROW 27:  ('pg_freespacemap',)
ROW 28:  ('pg_prewarm',)
ROW 29:  ('pg_stat_statements',)
ROW 30:  ('pg_surgery',)
ROW 31:  ('pg_trgm',)
ROW 32:  ('pg_visibility',)
ROW 33:  ('pgcrypto',)
ROW 34:  ('pgrowlocks',)
ROW 35:  ('pgstattuple',)
ROW 36:  ('plpgsql',)
ROW 37:  ('postgres_fdw',)
ROW 38:  ('refint',)
ROW 

In [8]:
db = PGEmbedding.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=collection_name,
    connection_string=connection_string,
)

query = "How has AWS evolved?"
#query = 'who is frank obama'
print()
print("-" * 80)
print(query)

#query = db.embedding_function.embed_query(query)
results_with_scores = db.similarity_search_with_score(query)

for doc, score in results_with_scores:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\nScore: {score}\n\n")

print()
print("-" * 80)


--------------------------------------------------------------------------------
How has AWS evolved?
Content: customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.5201462


Content: customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.5201462


Content: customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.
Metadata: {'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}
Score: 0.5202

In [ ]:
#https://platform.openai.com/docs/guides/text-generation

!pip install gradio --quiet
!pip install xformer --quiet
!pip install chromadb --quiet
!pip install langchain --quiet
!pip install accelerate --quiet
!pip install transformers --quiet
!pip install bitsandbytes --quiet
!pip install unstructured --quiet
!pip install sentence-transformers --quiet
!pip install pypdf

%pip install openai==0.28  --root-user-action=ignore
%pip install tiktoken

In [11]:
#!pip install accelerate

import torch
from textwrap import fill
from IPython.display import Markdown, display

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    )

from langchain import PromptTemplate
from langchain import HuggingFacePipeline

from langchain.vectorstores import Chroma
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredMarkdownLoader, UnstructuredURLLoader
from langchain.chains import LLMChain, SimpleSequentialChain, RetrievalQA, ConversationalRetrievalChain
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
import warnings
warnings.filterwarnings('ignore')

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024
generation_config.temperature = 0.8
generation_config.top_p = 0.95
generation_config.do_sample = True
generation_config.repetition_penalty = 1.15

pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
llm = HuggingFacePipeline(pipeline=pipeline,)

In [19]:
from langchain.llms import OpenAI
#import colab_env
from IPython.display import Markdown, display

from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

#retriever = db.as_retriever(search_type="similarity_score_threshold",search_kwargs='1')
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})

# create a chain to answer questions
#qa = RetrievalQA.from_chain_type(
#     llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=True)

qa = RetrievalQA.from_chain_type(
     llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

query = "How AWS has evolved?"
#query = "How many AI publications in 2022?"
result = llm(query)

#result = qa(query)
display(Markdown(f"<b>{query}</b>"))
display(Markdown(f"<p>{result}</p>"))

print()
print('chain to answer questions')
print("-" * 80)
result = qa({"query": query})
print(f'Query: {result["query"]}\n')
print(f'Result: {result["result"]}\n')
print(f'Context Documents: ')
for srcdoc in result["source_documents"]:
      print(f'{srcdoc}\n')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<b>How AWS has evolved?</b>

<p>

I've been working with the cloud for almost 10 years now, but I have never seen a company evolve so much. The way they keep innovating is amazing and it's clear that customer success is at the core of everything they do. If you look back just 5 or 6 years, AWS was still a single region provider and their pricing models were very complex. Today, AWS is truly global with more regions than any other cloud provider, and their pricing models are simpler and easier to understand. They have also invested heavily in developer tools and services to help developers build faster, cheaper, and better applications. Additionally, AWS has become more open and community-focused by launching initiatives such as Open Source Toolkit (OpenSTK) and AWS Lambda Layers. These new initiatives enable customers to share code and best practices within the AWS ecosystem, which helps drive innovation and enables customers to learn from each other. Overall, AWS has evolved from being just an infrastructure provider to becoming a true partner that helps customers innovate and transform their businesses.</p>


chain to answer questions
--------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Query: How AWS has evolved?

Result: 
It sounds like AWS has evolved from providing customers with more functionality than they could find elsewhere, which was already a significant differentiator. However, the statement suggests that this evolution led to even more groundbreaking and transformative offerings from AWS. Without more information, it's difficult to say exactly how AWS has evolved or what those game-changing offerings might be.

Context Documents: 
page_content='customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.' metadata={'year': 2021, 'source': 'AMZN-2021-Shareholder-Letter.pdf'}

page_content='customersmuch more functionality in AWS than they can find anywhere else (which is a significant differentiator), butalso allowed us to arrive at the much more game-changing offering that AWS is today.' metadata={'year': 2021, 'sour